*Copyright (c) Tumiz. Distributed under the terms of the GPL-3.0 License.*

# Position and Velocity

**Vector3**, represent point or position, velocity and other vectors. It is a class inheriting torch.Tensor, so it is also a tensor.

In [5]:
a=Vector3(1,2,2)
b=Vector3(1,-2,-0.8)
(a-b).norm()

tensor(4.8826)

In [1]:
from scenario import *
Vector3(1,2,-0.3)

tensor([ 1.0000,  2.0000, -0.3000], dtype=torch.float64)

In [2]:
Vector3(1,-2,3)*3

tensor([ 3., -6.,  9.], dtype=torch.float64)

In [3]:
Vector3(1,2,3)+Vector3(-1,-2,-3)

tensor([0., 0., 0.], dtype=torch.float64)

Calculate distance between two points.

In [4]:
point1=Vector3(1,2,3)
point2=Vector3(-10,87,11)
distance=(point1-point2).norm()
print(distance)

tensor(86.0814, dtype=torch.float64)


# Rotation

## Theory

Rotation is a physical quantity describing a process that object rotate some angle around an axis. It can be described in different form. Usual forms are
* eular angles, $(\alpha,\beta,\gamma),\rightarrow T(\alpha,\beta,\gamma)$ 
* axis-angle, $(\textbf{r},\theta),\rightarrow T(\textbf{r},\theta)$
* quaternion, $(x,y,z,w),\rightarrow T(x,y,z,w)$ 
* direction change, $(\textbf{d},\textbf{d}_c),\rightarrow T(\textbf{d},\textbf{d}_c)$

From those parameters, we can get rotation matrices, the matrices have different parameters but same value.

$T(\alpha,\beta,\gamma)=T(\textbf{r},\theta)=T(x,y,z,w)=T(\textbf{d},\textbf{d}_c)$

In [3]:
from sympy import *
def rx(s):
    x = symbols(s)
    m = Matrix([[1,0,0],[0,cos(x),-sin(x)],[0,sin(x),cos(x)]])
    return m
def ry(s):
    y=symbols(s)
    m = Matrix([[cos(y),0,sin(y)],[0,1,0],[-sin(y),0,cos(y)]])
    return m
def rz(z):
    z=symbols(z)
    m = Matrix([[cos(z),-sin(z),0],[sin(z),cos(z),0],[0,0,1]])
    return m
m=rx('x')*ry('y')*rz('z')
m

Matrix([
[                       cos(y)*cos(z),                        -sin(z)*cos(y),         sin(y)],
[sin(x)*sin(y)*cos(z) + sin(z)*cos(x), -sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(y)],
[sin(x)*sin(z) - sin(y)*cos(x)*cos(z),  sin(x)*cos(z) + sin(y)*sin(z)*cos(x),  cos(x)*cos(y)]])

In [2]:
from sympy import *
init_printing(use_unicode=True)
sx = symbols('s_x')
sy = symbols('s_y')
sz = symbols('s_z')
s=Matrix([
    [sx,0,0,0],
    [0,sy,0,0],
    [0,0,sz,0],
    [0,0,0,1]
         ])
s

⎡sₓ   0    0   0⎤
⎢               ⎥
⎢0   s_y   0   0⎥
⎢               ⎥
⎢0    0   s_z  0⎥
⎢               ⎥
⎣0    0    0   1⎦

## Definition

Rotation is a 4X4 **matrix**, which default value is a 4x4 identity matrix

In [3]:
from scenario import *
r=Rotation()
print("Matrix",r)
print("Eular angles",r.to_eular())
print("Quaternion",r.to_quaternion())
print("Axis-angle",r.to_axis_angle())

Matrix tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Eular angles [-0.0, 0.0, -0.0]
Quaternion [0.0, 0.0, 0.0, 1.0]
Axis-angle (tensor([0., 0., 0.]), 0.0)


A rotation can be used to represent angular velocity. A angular velocity 0.3rad/s around axis (0.1,0.2,0.3) can be represented like this:

### Eular angles

Scenario uses Tait-Bryan angles. Tait-Bryan rotations rotate about three distinct axes (x y z). Threre are two types, one is intrinsic rotation, and another is extrinsic. By default, Rotation.Eular(x,y,z) returns a intrinscis one in the order of 'XYZ'.

Intrinsic rotations apply to axis of body coordinate system. Coordinate system of next rotation is relative to previous rotation. That is, for order 'XYZ', the rotation is first around the local-X axis (which is the same as the world-X axis), then around local-Y (which may now be different from the world Y-axis), then local-Z (which may be different from the world Z-axis).

Extrinsic rotations apply to axis of world coordinate system. Coordinate system of next rotation is relative to (fixed) world coordinate system

#### Eg.1
Different ways to define a rotation by eular angles

In [8]:
from scenario import *
a=Rotation.Rx(0.3)*Rotation.Ry(0.1)*Rotation.Rz(-0.2) #order XYZ, intrinsic rotation
print(a==Rotation.Eular(0.3,0.1,-0.2,EularType.Intrinsic))
print(a.to_eular(EularType.Intrinsic))
b=Rotation.Rz(-0.2)*Rotation.Ry(0.1)*Rotation.Rx(0.3) #order XYZ, extrinsic rotation
print(b==Rotation.Eular(0.3,0.1,-0.2,EularType.Extrinsic))
print(b.to_eular(EularType.Extrinsic))

True
[0.2999999838863107, 0.09999999234768961, -0.19999998672073357]
True
[0.2999999838863107, 0.09999999234768961, -0.19999998672073357]


#### Eg.2
Default values

In [13]:
from scenario import *
r=Rotation.Eular()
print(r.to_eular())
r=Rotation.Eular(y=1)
print(r.to_eular())

#### Eg.3
If you define a extrinsic rotation. Don't forget to set type when you get them.

In [13]:
i=Rotation.Eular(0.1,0.2,0.3)
i_=Rotation.Eular(0.1,0.2,0.3,EularType.Intrinsic)
e=Rotation.Eular(0.1,0.2,0.3,EularType.Extrinsic)
print(i.to_eular())
print(i_.to_eular())
print(e.to_eular())
print(e.to_eular(EularType.Extrinsic))

[-0.0, 0.0, -0.0]
[-0.0, 0.9999999996174541, -0.0]
[0.09999999389744785, 0.1999999902005501, 0.29999999329756577]
[0.09999999389744785, 0.1999999902005501, 0.29999999329756577]
[0.03787987191831012, 0.22012402162886374, 0.28577168627635946]
[0.09999999389744785, 0.1999999902005501, 0.29999999329756577]


#### Eg.4

In [5]:
a=Rotation().rotate_x(0.1).rotate_z(0.2).rotate_y(-0.4)
b=Rotation().rotate_x(0.1).rotate_y(-0.4).rotate_y(0.2)
a==b

False

Two ways below to difine a rotation return the same value.

In [6]:
a=Rotation().rotate_x(0.1).rotate_y(0.2).rotate_z(-0.4)
b=Rotation.Eular(0.1,0.2,-0.4)
a==b

True

### Quaternion
Rotation.Quaternion() defines a rotation from quaternion. The form of quaternion must be (x,y,z,w)

#### Eg.1
First define a intrinsic rotation from eular angles

In [21]:
from scenario import *
r=Rotation.Eular(0.3,0.1,-0.2)
r.to_eular()

[0.2999999838863107, 0.09999999234768961, -0.19999998672073357]

Assign it to a cube

In [22]:
scen=Scenario()
cube=Cube()
cube.rotation=r
scen.add(cube)
scen.render()

Get its **quaternion** representation

In [23]:
r.to_quaternion()

[0.14357212505700329,
 0.06407121738575826,
 -0.09115751282474704,
 0.9833474624286825]

Assgin this quaternion to a new object

In [24]:
q=Cube()
q.position+=Vector3(2,0,0)
scen.add(q)
q.rotation=Rotation.Quaternion(0.14357212505700329,
 0.06407121738575826,
 -0.09115751282474704,
 0.9833474624286825)
scen.render()

### Axis-angle

Get **axis-angle** representation

In [7]:
r.to_axis_angle()

(tensor([ 0.7900,  0.3526, -0.5016]), 0.36550197536514045)

Assign this axis-angle to a new object

In [8]:
a=Cube()
a.position+=Vector3(4,0,0)
axis=torch.tensor([ 0.8092,  0.1807, -0.5591])
a.rotation=Rotation.Axis_angle(axis,0.38156478417971545)
scen.add(a)
scen.render()

### Direction change

In [6]:
from scenario import *
scen=Scenario()
c=Cube()
print(c.direction)
c.rotation=Rotation.Direction_change(c.direction,Vector3(1,1,1))
print(c.rotation.to_axis_angle())
print(c.rotation.to_eular())
d=Line.Vector(c.direction)
c.add(d)
scen.add(c)
scen.render()

tensor([1., 0., 0.])
(tensor([ 0.0000, -0.7071,  0.7071]), 0.9553166308158254)
[0.2617994131992246, -0.6154797142584597, 0.7853981633974483]


An object's rotation can be changed by set its **lookat** point.

In [5]:
from scenario import *
scen=Scenario()
cube=Cube()
cube.scale=Vector3(2,1,1)
cube.lookat(Vector3(1,1,1))
cube.color=Color(r=1)
cube.add(Line.Vector(cube.direction))
scen.add(cube)
scen.render()

Each object has a local vector to represent its face **direction**. You can get and set it directly.

In [2]:
print(cube.direction)
cube.direction=Vector3(0,1,0)
print(cube.direction)

tensor([1., 0., 0.])
tensor([0., 1., 0.])


Show the direction by a **Vector**

In [3]:
d=Line.Vector(cube.direction*3)
cube.add(d)
scen.render()

## Copy

Rotation.clone() will return a deep **copy** of original rotation

In [2]:
from scenario import *
r=Rotation()
c=r.clone()
print(id(r),id(c))
print(r==c)

140176395793584 140178341024544
True


## Compare
Use == to **compare** two rotations, it will tell if those two ratation are the same.

In [13]:
from scenario import *
a=Rotation.Eular(0,0.1,0.3)
b=Rotation.Axis_angle(Vector3(1,1,1),2)
print(a*b==b*a)
print(a*b)
print(b*a)

False
tensor([[-0.2466,  0.0283,  0.9687,  0.0000],
        [ 0.9674,  0.0673,  0.2443,  0.0000],
        [-0.0583,  0.9973, -0.0439,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
tensor([[-0.0620, -0.0671,  0.9958,  0.0000],
        [ 0.9695, -0.2412,  0.0441,  0.0000],
        [ 0.2373,  0.9681,  0.0800,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


In [5]:
type(b)

scenario.main.Rotation

## Multiply

### Rotation*Rotation

Assume a object rotated twice, first time rotated A, and the second time B, both A and B are represented in parent space, A\*B returns the total rotation in parent space. If A is represented in parent space, but B is in local space, then B\*A returns the toal rotation in parent space.

$r_{current}=r_{origin}r_{parent}$, rotate $r_{parent}$ 

$r_{current}=r_{local}r_{origin}$, rotate $r_{local}$

$r_{origin}^{-1}r_{current}=r_{origin}^{-1}r_{origin}r_{parent}$

$r_{origin}^{-1}r_{current}=r_{parent}$

$r_{origin}^{-1}r_{local}r_{origin}=r_{parent}$

Rotate $r_{parent}$

In [14]:
from scenario import *
scen=Scenario()
xyz=XYZ()
xyz.position=Vector3(1,1,1)
xyz.rotation=Rotation.Eular(0,0.5,0)
scen.add(xyz)
scen.render()

In [9]:
xyz1=XYZ()
xyz1.position=xyz.position
xyz1.rotation=xyz.rotation*Rotation.Eular(0,0,0.3)
print(xyz.id,xyz1.id)
scen.add(xyz1)
scen.render()

0 1


Rotate $r_{local}$

In [1]:
from scenario import *
scen=Scenario()
xyz=XYZ()
xyz.position=Vector3(1,1,1)
xyz.rotation=Rotation.Eular(0,0.5,0)
plane=Cube(5,5,0.001)
plane.position=xyz.position.clone()
plane.rotation=xyz.rotation.clone()
scen.add(xyz,plane)

xyz.rotation=Rotation.Eular(0,0,0.3)*xyz.rotation
scen.render()

### Rotation*float/int

Since a rotation can be used to represent angular velocity, so a rotation multiply a float or integer means a angular velocity multiply a time duration. If we define a rotation and let it multiply a float, we will see the result has a same axis as the rotation.

In [28]:
from scenario import *
a=Rotation.Eular(0.1,0.2,0.3)
print(a.to_axis_angle())
b=a*0.1
print(b.to_axis_angle())

(tensor([0.1886, 0.5834, 0.7900], dtype=torch.float64), 0.36550218635669895)
(tensor([0.1886, 0.5834, 0.7900], dtype=torch.float64), 0.03655021863566978)


### Rotation*Vector3

means convert a local vector to parent space.

In [1]:
from scenario import *
scen=Scenario()
local=Vector3(1,1,1)
local_v=Line.Vector(local)
xyz=XYZ()
xyz.position=Vector3(1,2,3)
xyz.rotation=Rotation.Eular(0,1,-1)
xyz.add(local_v)
scen.add(xyz)
scen.render()

In [2]:
world=xyz.rotation*local
world_v=Line.Vector(world)
scen.add(world_v)
scen.render()

In [2]:
from scenario import *
r=Rotation.Eular(0.1,0.2,0.3)
axis=Vector3(1,2,3)
angle=0.3
axis_world=r*axis
r_=r*Rotation.Axis_angle(axis,angle)
r_1=Rotation.Axis_angle(axis_world,angle)*r
print(r_.to_axis_angle())
print(r_1.to_axis_angle())
print(r_)
print(r_1)

(tensor([0.3032, 0.5002, 0.8111]), 0.6808914515633198)
(tensor([0.3032, 0.5002, 0.8111]), 0.6808914515633198)
tensor([[ 0.7975, -0.4768,  0.3697,  0.0000],
        [ 0.5444,  0.8328, -0.1004,  0.0000],
        [-0.2600,  0.2813,  0.9237,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
tensor([[ 0.7975, -0.4768,  0.3697,  0.0000],
        [ 0.5444,  0.8328, -0.1004,  0.0000],
        [-0.2600,  0.2813,  0.9237,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


In [7]:
from scenario import *
r=Rotation.Eular(0,0,0.6)
loc_av=Rotation.Eular(0,1,0.1)
av=loc_av*r
print(av.to_axis_angle())
da=av*0.1
print(da.to_axis_angle())

(tensor([0.2095, 0.8206, 0.5316]), 1.2033908931164188)
(tensor([0.2095, 0.8206, 0.5316]), 0.12033855958633025)


In [8]:
r=Rotation.Eular(0,0,0.6)
loc_av=Rotation.Eular(0,1,0.1)
loc_da=loc_av*0.1
print(loc_da.to_axis_angle())
da=loc_da*r
print(da.to_axis_angle())

(tensor([-0.0498,  0.9946,  0.0911]), 0.1004561780627073)
(tensor([0.0407, 0.1595, 0.9864]), 0.6170671260140874)


In [4]:
v0=r*Rotation.Axis_angle(axis,angle)*r.T()
v1=Rotation.Axis_angle(axis_world,angle)
print(v0.matrix)
print(v1.matrix)

tensor([[ 0.9588, -0.2239,  0.1749,  0.0000],
        [ 0.2378,  0.9693, -0.0627,  0.0000],
        [-0.1555,  0.1017,  0.9826,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)
tensor([[ 0.9588, -0.2239,  0.1749,  0.0000],
        [ 0.2378,  0.9693, -0.0627,  0.0000],
        [-0.1555,  0.1017,  0.9826,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)


In [17]:
import torch
v=torch.rand(4,4)
print(torch.inverse(v).mm(v),v.mm(v.inverse()))
print(v.T.mm(v))
print(v.t().mm(v))

tensor([[ 1.0000e+00,  5.9605e-08, -1.1921e-07, -1.1921e-07],
        [ 0.0000e+00,  1.0000e+00, -2.3842e-07, -2.3842e-07],
        [-2.3842e-07,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 5.5879e-09,  0.0000e+00,  0.0000e+00,  1.0000e+00]]) tensor([[ 1.0000e+00, -2.9802e-08, -2.3842e-07,  2.3842e-07],
        [-5.9605e-08,  1.0000e+00, -4.7684e-07,  4.7684e-07],
        [ 2.9802e-08,  5.9605e-08,  1.0000e+00,  1.7881e-07],
        [ 0.0000e+00,  8.9407e-08, -5.9605e-08,  1.0000e+00]])
tensor([[1.6088, 0.5215, 1.4131, 1.4138],
        [0.5215, 0.7669, 1.1717, 0.5581],
        [1.4131, 1.1717, 2.1671, 1.3238],
        [1.4138, 0.5581, 1.3238, 1.8585]])
tensor([[1.6088, 0.5215, 1.4131, 1.4138],
        [0.5215, 0.7669, 1.1717, 0.5581],
        [1.4131, 1.1717, 2.1671, 1.3238],
        [1.4138, 0.5581, 1.3238, 1.8585]])


## Transform

In [2]:
from scenario import *
t=Transform()
print(t.position)
print(t.rotation)

tensor([0., 0., 0.])
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [6]:
from scenario import *
a=Transform()
p=a.world_position()
r=a.world_rotation()
print(a.world_position())
print(r*r,type(r*r))
print(a.rotation,type(a.rotation))
print(type(a.rotation*a.rotation))

tensor([0., 0., 0.])
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]) <class 'scenario.main.Rotation'>
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]) <class 'scenario.main.Rotation'>
<class 'scenario.main.Rotation'>


**Local position to world**

In [1]:
from scenario import *
scen=Scenario()
cube=Cube()
cube.color=Color(r=1)
cube.lookat(Vector3(1,2,3))
cube.scale=Vector3(2,1,1)
cube.position=Vector3(0,-1,3)

point=Sphere()
point.position=cube.rotation*(cube.scale*Vector3(-0.5,0.5,0.5))+cube.position
point.radius=0.1
point.color=Color(b=1)
scen.add(cube,point)
scen.render()

In [7]:
scen.add(Line.Vector(Vector3(-0.5,0.5,0.5)))
scen.render()

In [8]:
scen.add(Line.Vector(cube.scale*Vector3(-0.5,0.5,0.5)))
scen.render()